In [1]:
import numpy as np



In [2]:
def project_on_direction(X, coef, intercept):
    """
    Projects the data X onto the hyperplane defined by coef and intercept.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input data to be projected.
    coef : array-like, shape (n_features,)
        The coefficients defining the hyperplane.
    intercept : float
        The intercept defining the hyperplane.

    Returns
    -------
    projections : array, shape (n_samples,)
        The projections of each sample onto the hyperplane.
    """
    # Calculate the norm of the coefficient vector
    norm_coef = np.linalg.norm(coef)

    # Project each sample onto the hyperplane
    projections = (np.dot(X, coef) + intercept) / norm_coef

    return projections
